(meter-analysis)=
# Meter analysis

In [ ]:
## Importing compiam to the project
import compiam

# Import extras and supress warnings to keep the tutorial clean
import os
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# We import the tool
from compiam.rhythm.meter import AksharaPulseTracker

# Let's initialize an instance of the tool
apt = AksharaPulseTracker()

In [ ]:
cmr = compiam.load_dataset(
    "compmusic_carnatic_rhythm",
    data_home=os.path.join("../audio/mir_datasets"),
    version="full_dataset"
)
cmr

For showcasing purposes, we include a single-track version of the [(CompMusic) Carnatic Rhythm Dataset](https://zenodo.org/record/1264394) within the materials of this tutorial. This dataset is private, but can be requested and downloaded for research purposes.

Reading through the dataset details we observe the available data for the tracks in the dataloader. We will load the tracks and select the specific track we have selected for tutoring purposes. Bear in mind that you can access the list of identifiers in the dataloader with the ``.track_ids`` attribute.

In [ ]:
track = cmr.load_tracks()["10001"]

Let's print out the available annotations for this example track.

In [ ]:
track.beats

In [ ]:
track.meter

We can now extract the beats (referred as akshara pulses within the context of Indian Art Music) from the input recording. Likewise the other extractors and models in `compiam` (if not indicated otherwise), the method for inference takes an audio path as input.

In [ ]:
predicted_beats = apt.extract(track.audio_path)

In [ ]:
predicted_beats

In [ ]:
from compiam.visualisation.audio import plot_waveform
plot_waveform(
    path_to_audio=track.audio_path,
    t1=0,
    t2=20,
    labels=predicted_beats,
)